In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Load the test data. Parse column time as time and use row id as index

In [ ]:
df=pd.read_csv('/kaggle/input/tabular-playground-series-mar-2022/train.csv',parse_dates=['time'],index_col=['row_id'])

In [ ]:
df.head()

In [ ]:
df.isna().sum()

We see that there are no missing values

In [ ]:
index_series=pd.DataFrame(df.time.drop_duplicates().sort_values().reset_index(drop=True))
index_series['times_gap']=index_series - index_series.shift(1)
index_series.sort_values(by='times_gap',ascending=False).head(5)

In [ ]:
sns.lineplot(data=index_series,y='times_gap',x='time')

We see that there are some missing values in the data. Expecially towards September-October 1991.

In [ ]:
print(df['x'].unique())
print(df['y'].unique())

We see that there are only 3 values for x and 4 for y.

In [ ]:
df['hour']=df.time.dt.hour
df['minute']=df.time.dt.minute
df['year']=df.time.dt.year
df['dayofyear']=df.time.dt.dayofyear

In [ ]:
sns.scatterplot(data=df,x='x',y='y')

In [ ]:
df['direction'].value_counts()

In [ ]:
visu=df.groupby(['x','y','direction','hour'])['congestion'].mean().reset_index()

In [ ]:
fig = plt.figure(figsize=(20, 50))

rows, cols = 10, 4
counter=1
for x in [0,1,2]:
    for y in [0,1,2,3]:
        ax = fig.add_subplot(rows, cols,counter)
        
        ax.grid(alpha = 0.7, axis ="both")
        #for  num in orientations:
        sns.lineplot(data=visu[(visu['x']==x)&(visu['y']==y)],y='congestion',x='hour',hue='direction',hue_order=['EB','WB', 'NB','SB', 'NE', 'SW', 'NW', 'SE'],ax=ax)
        ax.set_title(f'Point ({x},{y})')
        plt.xticks(fontsize=14)
        plt.yticks(fontsize=14)
        counter+=1
fig.tight_layout(h_pad=2)

Plotting the congestion at the various points over the day,

# Predictions

In [ ]:
df.head()

In [ ]:
median=df.groupby(['hour','minute','x','y','direction']).median()

In [ ]:
test=pd.read_csv('/kaggle/input/tabular-playground-series-mar-2022/test.csv',parse_dates=['time'])
test['hour']=test.time.dt.hour
test['minute']=test.time.dt.minute

In [ ]:
test=test.merge(median,how='left',on=['x','y','direction','hour','minute'])

In [ ]:
test[['row_id','congestion']].to_csv('submission.csv',index=False)